## openASSA Python proof of concept
1. Use custom accessor on pandas dataframes to add custom methods to standardize and validate input data
2. Define common custom calculations to apply to dataframe

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

custom accessor on pandas dataframes to create a namespace that allows validation to be performed

In [2]:
@pd.api.extensions.register_dataframe_accessor("lifetables")
class LifeAccessor:
    def __init__(self, pandas_obj):
        self._obj = pandas_obj
    
    def percent_to_decimal(self, column_name):
        if pd.api.types.is_string_dtype(self._obj[column_name]):
            self._obj[column_name] = self._obj[column_name].str.rstrip('%').astype('float') / 100.0
        return self._obj
        
    def validate(self, table_type):
        if table_type == 'life':
            if 'qx' not in self._obj.columns:
                raise AttributeError("Must have 'qx' column.")
            elif not pd.api.types.is_numeric_dtype(self._obj['qx']):
                raise AttributeError("This column must contain numeric data.")
            else:
                return True
        elif table_type == 'lapse':
            if 'lapse_rate' not in self._obj.columns:
                raise AttributeError("Must have 'lapse_rate' column.")
            elif not pd.api.types.is_numeric_dtype(self._obj['lapse_rate']):
                raise AttributeError("This column must contain numeric data.")
            else:
                return True
        elif table_type == 'yield':
            if 'yield_curve' not in self._obj.columns:
                raise AttributeError("Must have 'yield_curve' column.")
            elif not pd.api.types.is_numeric_dtype(self._obj['yield_curve']):
                raise AttributeError("This column must contain numeric data.")
            else:
                return True
        else:
            print('Provide a table type to validate: life, lapse or yield')
            return False

load the external data into dataframes, assuming tab delimited text files, could be exchanged for any other data source

In [3]:
life_df = pd.read_table('Mortality_Table.txt', index_col='Age')
lapse_df = pd.read_table('Lapse_Table.txt', index_col='Duration(Years)')
yield_df = pd.read_table('Yield_Curve.txt', index_col='Duration(Months)')

validate the dataframes

In [4]:
life_df.lifetables.validate('life')
lapse_df.lifetables.validate('lapse')
yield_df.lifetables.validate('yield')

AttributeError: Must have 'lapse_rate' column.

In [5]:
lapse_df.head()

,Annual_Lapse_Rate
Duration(Years),
1,10%
2,9%
3,8%
4,7%
5,6%


In [6]:
list(lapse_df)

['Annual_Lapse_Rate']

In [7]:
lapse_df.rename(columns={'Annual_Lapse_Rate': 'lapse_rate'}, inplace=True)
lapse_df.columns

Index(['lapse_rate'], dtype='object')

In [8]:
lapse_df.lifetables.validate('lapse')

AttributeError: This column must contain numeric data.

In [9]:
lapse_df = lapse_df.lifetables.percent_to_decimal('lapse_rate')
lapse_df.lifetables.validate('lapse')
lapse_df

,lapse_rate
Duration(Years),
1,0.10
2,0.09
3,0.08
4,0.07
5,0.06
6,0.05
7,0.04
8,0.03
9,0.02


In [10]:
#yield_df.lifetables.validate('yield') 
yield_df.rename(columns={'Spot_Rate(NACA)': 'yield_curve'}, inplace=True)
yield_df.lifetables.percent_to_decimal('yield_curve')
yield_df.lifetables.validate('yield')

True

create period indices

In [11]:
def create_period_index(dataframe, freq, start=datetime.today().date()):
    dataframe.index = pd.date_range(start=start, periods=len(dataframe), freq=freq).to_period()
    return dataframe

In [12]:
yield_df = create_period_index(yield_df, 'M')
lapse_df = create_period_index(lapse_df, 'Y')

convert yearly to monthly

In [13]:
yield_df

,yield_curve
2020-10,0.1
2020-11,0.1
2020-12,0.1
2021-01,0.1
2021-02,0.1
...,...
2072-04,0.1
2072-05,0.1
2072-06,0.1
2072-07,0.1


In [14]:
lapse_df = (lapse_df / 12).resample('M').interpolate()

simple life class with validation rules built in

In [19]:
class simple_life():
    '''
    docstring goes here
    '''

    def __init__(self, date_of_birth, premium, sum_assured, months_in_force=0):
        '''
        :param date_of_birth, must be a datetime object
        :param premium: the premium paid per period
        :param sum_assured: the sum assured
        :param months_in_force: total months the policy has been in force
        '''
        self.date_of_birth = date_of_birth
        self.premium = premium
        self.sum_assured = sum_assured
        self.months_in_force = months_in_force
        
    def calculate_deaths(self, qx, lapse_rate):
        return qx / 12 * (1 - 0.5 * lapse_rate)

    def calculate_surrenders(self, qx, lapse_rate):
        return lapse_rate * (1 - 0.5 * lapse_rate)
        
    def calculate_age(self, date):
        return date.year - self.date_of_birth.year - ((date.month, date.day) < (self.date_of_birth.month, self.date_of_birth.day))

    def forecast_table(self, forecast_periods, life_df, lapse_df, yield_df, freq='M'):
        '''
        :param forecast_periods: the policyholder's age in years
        :param frequency: the premium paid per period
         '''
        valid_life = life_df.lifetables.validate('life')
        valid_lapse = lapse_df.lifetables.validate('lapse')
        valid_yield = yield_df.lifetables.validate('yield')
        
        lapse_df.index = lapse_df.index - self.months_in_force
        forecast_index = pd.date_range(start=datetime.today().date(), periods=forecast_periods, freq=freq).to_period()
        
        self.forecast_df = pd.DataFrame(index=forecast_index)
        self.forecast_df['row_number'] = range(len(self.forecast_df))
        self.forecast_df['months_in_force'] = self.forecast_df['row_number'] + self.months_in_force
        self.forecast_df['age'] = self.forecast_df.index.map(self.calculate_age)
        self.forecast_df['qx'] = self.forecast_df.merge(life_df, left_on='age', right_index=True)['qx']        
        self.forecast_df['lapse_rate'] = self.forecast_df.merge(lapse_df, left_index=True, right_index=True)['lapse_rate']
        self.forecast_df['deaths'] = self.forecast_df.apply(lambda x: self.calculate_deaths(x.qx, x.lapse_rate) / 12, axis=1)
        self.forecast_df['surrenders'] = self.forecast_df.apply(lambda x: self.calculate_surrenders(x.qx, x.lapse_rate), axis=1)
        self.forecast_df['active_policies'] = 1 - (self.forecast_df['deaths'].cumsum() + self.forecast_df['deaths'].cumsum())
        self.forecast_df['premium_income'] = self.forecast_df['active_policies'] * self.premium
        self.forecast_df['claims_outgo'] = self.forecast_df['deaths'] * self.sum_assured
        self.forecast_df['profit'] = self.forecast_df['premium_income'] - self.forecast_df['claims_outgo']
        self.forecast_df['yield_curve'] = self.forecast_df.merge(yield_df, left_index=True, right_index=True)['yield_curve']
        self.forecast_df['npv_of_profit'] = self.forecast_df['profit'] * (1 + self.forecast_df['yield_curve']) ** (self.forecast_df['row_number'])

In [20]:
policy = simple_life(pd.Timestamp('1992-12-20'), 3600, 1000000, 5)
policy.forecast_table(5, life_df, lapse_df, yield_df, 'M')
policy.forecast_df

,row_number,months_in_force,age,qx,lapse_rate,deaths,surrenders,active_policies,premium_income,claims_outgo,profit,yield_curve,npv_of_profit
2020-10,0,5,27,0.00292,0.007014,0.00002,0.006989,0.999960,3599.854512,20.206665,3579.647847,0.1,3579.647847
2020-11,1,6,27,0.00292,0.006944,0.00002,0.006920,0.999919,3599.709019,20.207369,3579.501650,0.1,3937.451815
2020-12,2,7,28,0.00287,0.006875,0.00002,0.006851,0.999879,3599.566012,19.862044,3579.703968,0.1,4331.441801
2021-01,3,8,28,0.00287,0.006806,0.00002,0.006782,0.999840,3599.423001,19.862736,3579.560264,0.1,4764.394712
2021-02,4,9,28,0.00287,0.006736,0.00002,0.006713,0.999800,3599.279984,19.863428,3579.416556,0.1,5240.623779
